In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("100 Sales Records.csv")
print("First 5 rowes of the dataset:")
print(df.head())
df['Order Date']=pd.to_datetime(df['Order Date'])
df['Ship Date']=pd.to_datetime(df['Ship Date'])
print("\n Missing Values in Each Column:")
print(df.isnull().sum())
print("\n Data Types of Columns:")
print(df.dtypes)



Saving 100 Sales Records.csv to 100 Sales Records.csv
First 5 rowes of the dataset:
                              Region                Country        Item Type  \
0              Australia and Oceania                 Tuvalu        Baby Food   
1  Central America and the Caribbean                Grenada           Cereal   
2                             Europe                 Russia  Office Supplies   
3                 Sub-Saharan Africa  Sao Tome and Principe           Fruits   
4                 Sub-Saharan Africa                 Rwanda  Office Supplies   

  Sales Channel Order Priority Order Date   Order ID  Ship Date  Units Sold  \
0       Offline              H  5/28/2010  669165933  6/27/2010        9925   
1        Online              C  8/22/2012  963881480  9/15/2012        2804   
2       Offline              L   5/2/2014  341417157   5/8/2014        1779   
3        Online              C  6/20/2014  514321792   7/5/2014        8102   
4       Offline              L   2/1/201

In [1]:
import pandas as pd
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Load the data
df = pd.read_csv('/content/ecommerce_delivery.csv')

# Display basic info about the data
print("Dataset Info:")
print(df.info())
print("\nFirst few rows:")
print(df.head())

# 1. Create a basic delivery locations map
def create_delivery_locations_map(df):
    """Create a map showing all delivery locations"""
    # Calculate center of all points for initial map view
    center_lat = df['Latitude'].mean()
    center_lon = df['Longitude'].mean()

    # Create base map
    delivery_map = folium.Map(location=[center_lat, center_lon], zoom_start=5)

    # Add markers for each delivery location
    for idx, row in df.iterrows():
        # Color code markers based on delivery delay
        if row['Delivery_Delay'] == 0:
            color = 'green'
            popup_text = f"Order {row['Order_ID']}: On Time ⭐"
        elif row['Delivery_Delay'] == 1:
            color = 'orange'
            popup_text = f"Order {row['Order_ID']}: Slight Delay"
        else:
            color = 'red'
            popup_text = f"Order {row['Order_ID']}: Significant Delay"

        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=popup_text,
            tooltip=f"Order: {row['Order_ID']}, Delay: {row['Delivery_Delay']} days",
            icon=folium.Icon(color=color, icon='shopping-cart', prefix='fa')
        ).add_to(delivery_map)

    return delivery_map

# 2. Create delivery routes by date
def create_delivery_routes_by_date(df):
    """Create maps showing delivery routes organized by date"""

    # Group by delivery date
    dates = df['Delivery_Date'].unique()

    for date in dates:
        date_data = df[df['Delivery_Date'] == date]

        if len(date_data) > 1:
            # Create map for this date
            center_lat = date_data['Latitude'].mean()
            center_lon = date_data['Longitude'].mean()

            date_map = folium.Map(location=[center_lat, center_lon], zoom_start=6)

            # Add title
            title_html = f'''
                     <h3 align="center" style="font-size:20px"><b>Delivery Routes - {date}</b></h3>
                     '''
            date_map.get_root().html.add_child(folium.Element(title_html))

            # Create route lines connecting delivery points
            locations = list(zip(date_data['Latitude'], date_data['Longitude']))
            folium.PolyLine(
                locations,
                weight=3,
                color='blue',
                opacity=0.7,
                popup=f'Delivery Route {date}',
                tooltip=f'Click for route details'
            ).add_to(date_map)

            # Add markers for each delivery
            for idx, row in date_data.iterrows():
                folium.Marker(
                    location=[row['Latitude'], row['Longitude']],
                    popup=f"Order {row['Order_ID']}",
                    tooltip=f"Value: ${row['Order_Value']}, Delay: {row['Delivery_Delay']}d",
                    icon=folium.Icon(color='purple', icon='home')
                ).add_to(date_map)

            # Save map for this date
            date_map.save(f'delivery_routes_{date}.html')
            print(f"Created map for {date}: delivery_routes_{date}.html")

# 3. Create heatmap of delivery density
def create_delivery_heatmap(df):
    """Create a heatmap showing delivery density"""
    center_lat = df['Latitude'].mean()
    center_lon = df['Longitude'].mean()

    heatmap = folium.Map(location=[center_lat, center_lon], zoom_start=5)

    # Add heatmap layer
    heat_data = [[row['Latitude'], row['Longitude']] for idx, row in df.iterrows()]
    plugins.HeatMap(heat_data, radius=15, blur=10, gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(heatmap)

    # Add title
    title_html = '''
             <h3 align="center" style="font-size:20px"><b>Delivery Density Heatmap</b></h3>
             '''
    heatmap.get_root().html.add_child(folium.Element(title_html))

    return heatmap

# 4. Create clustered map view
def create_clustered_map(df):
    """Create a map with clustered markers"""
    center_lat = df['Latitude'].mean()
    center_lon = df['Longitude'].mean()

    cluster_map = folium.Map(location=[center_lat, center_lon], zoom_start=5)

    # Create marker cluster
    marker_cluster = plugins.MarkerCluster().add_to(cluster_map)

    # Add markers to cluster
    for idx, row in df.iterrows():
        # Determine icon color based on delivery performance
        if row['Delivery_Delay'] == 0:
            icon_color = 'green'
            icon_text = '✓'
        elif row['Delivery_Delay'] == 1:
            icon_color = 'orange'
            icon_text = '~'
        else:
            icon_color = 'red'
            icon_text = '✗'

        popup_content = f"""
        <b>Order {row['Order_ID']}</b><br>
        Customer: {row['Customer_ID']}<br>
        Value: ${row['Order_Value']}<br>
        Delivery Time: {row['Delivery_Time']} days<br>
        Delay: {row['Delivery_Delay']} days<br>
        Feedback: {row['Feedback']}
        """

        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=f"Order {row['Order_ID']} - {icon_text}",
            icon=folium.Icon(color=icon_color, icon='info-sign')
        ).add_to(marker_cluster)

    return cluster_map

# 5. Create delivery performance by region analysis
def analyze_regional_performance(df):
    """Analyze and visualize delivery performance by geographic regions"""

    # Create a simple regional classification based on coordinates
    def get_region(lat, lon):
        if lat > 20 and lon > 80:
            return 'East'
        elif lat > 20 and lon <= 80:
            return 'West'
        elif lat <= 20 and lon > 80:
            return 'South'
        else:
            return 'Central'

    df['Region'] = df.apply(lambda row: get_region(row['Latitude'], row['Longitude']), axis=1)

    # Calculate regional performance metrics
    regional_stats = df.groupby('Region').agg({
        'Delivery_Delay': 'mean',
        'Order_Value': 'mean',
        'Delivery_Time': 'mean',
        'Order_ID': 'count'
    }).rename(columns={'Order_ID': 'Order_Count'})

    print("\nRegional Performance Analysis:")
    print(regional_stats)

    # Create regional performance map
    region_map = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)

    # Color regions based on average delay
    region_colors = {
        'East': 'red' if regional_stats.loc['East', 'Delivery_Delay'] > 1 else 'orange',
        'West': 'red' if regional_stats.loc['West', 'Delivery_Delay'] > 1 else 'orange',
        'South': 'red' if regional_stats.loc['South', 'Delivery_Delay'] > 1 else 'orange',
        'Central': 'red' if regional_stats.loc['Central', 'Delivery_Delay'] > 1 else 'orange'
    }

    # Add regional markers
    for region in df['Region'].unique():
        region_data = df[df['Region'] == region]
        center_lat = region_data['Latitude'].mean()
        center_lon = region_data['Longitude'].mean()

        folium.Marker(
            location=[center_lat, center_lon],
            popup=f"""
            <b>{region} Region</b><br>
            Avg Delay: {regional_stats.loc[region, 'Delivery_Delay']:.2f} days<br>
            Avg Order Value: ${regional_stats.loc[region, 'Order_Value']:.2f}<br>
            Total Orders: {regional_stats.loc[region, 'Order_Count']}
            """,
            icon=folium.Icon(color=region_colors[region], icon='globe')
        ).add_to(region_map)

    return region_map, regional_stats

# Execute all visualizations
print("Creating geo-spatial visualizations...")

# 1. Basic delivery locations map
print("1. Creating basic delivery locations map...")
basic_map = create_delivery_locations_map(df)
basic_map.save('delivery_locations.html')
print("Saved: delivery_locations.html")

# 2. Delivery routes by date
print("\n2. Creating delivery routes by date...")
create_delivery_routes_by_date(df)

# 3. Heatmap
print("\n3. Creating delivery density heatmap...")
heatmap = create_delivery_heatmap(df)
heatmap.save('delivery_heatmap.html')
print("Saved: delivery_heatmap.html")

# 4. Clustered map
print("\n4. Creating clustered delivery map...")
cluster_map = create_clustered_map(df)
cluster_map.save('delivery_clusters.html')
print("Saved: delivery_clusters.html")

# 5. Regional analysis
print("\n5. Analyzing regional performance...")
region_map, regional_stats = analyze_regional_performance(df)
region_map.save('regional_performance.html')
print("Saved: regional_performance.html")

# Create a summary dashboard map
print("\n6. Creating summary dashboard...")
def create_summary_dashboard(df):
    """Create a comprehensive dashboard map"""
    dashboard = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)

    # Add different layers
    # Heatmap layer
    heat_data = [[row['Latitude'], row['Longitude']] for idx, row in df.iterrows()]
    plugins.HeatMap(heat_data, radius=15, name='Delivery Density').add_to(dashboard)

    # Marker cluster layer
    marker_cluster = plugins.MarkerCluster(name='Delivery Points').add_to(dashboard)
    for idx, row in df.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"Order {row['Order_ID']}",
            icon=folium.Icon(color='blue', icon='shopping-cart')
        ).add_to(marker_cluster)

    # Add layer control
    folium.LayerControl().add_to(dashboard)

    # Add title
    title_html = '''
             <h3 align="center" style="font-size:20px"><b>E-commerce Delivery Dashboard</b></h3>
             '''
    dashboard.get_root().html.add_child(folium.Element(title_html))

    return dashboard

dashboard = create_summary_dashboard(df)
dashboard.save('delivery_dashboard.html')
print("Saved: delivery_dashboard.html")

print("\n" + "="*50)
print("ALL VISUALIZATIONS COMPLETED!")
print("="*50)
print("\nGenerated Files:")
print("1. delivery_locations.html - Basic delivery points map")
print("2. delivery_routes_YYYY-MM-DD.html - Route maps for each date")
print("3. delivery_heatmap.html - Delivery density heatmap")
print("4. delivery_clusters.html - Clustered delivery points")
print("5. regional_performance.html - Regional analysis map")
print("6. delivery_dashboard.html - Comprehensive dashboard")

print("\nOpen these HTML files in your browser to view the interactive maps!")

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order_ID        40 non-null     int64  
 1   Customer_ID     40 non-null     object 
 2   Order_Value     40 non-null     int64  
 3   Delivery_Time   40 non-null     int64  
 4   Delivery_Delay  40 non-null     int64  
 5   Feedback        40 non-null     object 
 6   Latitude        40 non-null     float64
 7   Longitude       40 non-null     float64
 8   Delivery_Date   40 non-null     object 
dtypes: float64(2), int64(4), object(3)
memory usage: 2.9+ KB
None

First few rows:
   Order_ID Customer_ID  Order_Value  Delivery_Time  Delivery_Delay  \
0      1001        C001          250              3               0   
1      1002        C002          480              5               1   
2      1003        C003          320              2               0   
3      1004        C0